In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers, losses, metrics
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_X(datasets_file: str, sample_length: int):
    with open(datasets_file, "r") as file:
        records = [line.rstrip().split() for line in file]
    X = []
    for line in records:
        n = len(line)
        if n >= sample_length:
            # print(line)
            for i in range(0, n - sample_length + 1, 2):
                # print(f"--> i={i} : {line[i : i + sample_length]}")
                X.append(
                    np.array(
                        [float(f"{float(x):.4f}") for x in line[i : i + sample_length]]
                    )
                )
        # else:
        #     x = line[0]
        #     X.append(np.array([np.float32(x), np.float32(np.random.uniform(1.0, 4.0)), np.float32(x)]))
    X = np.vstack(X)
    return X


X = get_X("datasets.txt", 3)
X.shape